<a href="https://colab.research.google.com/github/rathull/cbs-minihack-2023/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize Notebook

In [1]:
!unzip data_small.zip

Archive:  data_small.zip
  inflating: accessible_small.fasta  
  inflating: notaccessible_small.fasta  


In [2]:
# https://www.kaggle.com/code/chuckzzzz/dna-classification-with-deep-learning-part-1

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import random
import pickle
import matplotlib.pyplot as plt

import torch.utils.data as data_utils
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import nn, optim, utils
from torchvision import datasets, transforms
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from torchsummary import summary

In [3]:
# use GPU is GPU (CUDA) is available, use CPU otherwise
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


# Generate k mer dataset

In [4]:
# Split seq to kmers

def kmer_split(seq, size=5):
    kmers = []
    for x in range(len(seq) - size + 1):
        kmers.append(seq[x:x+size].lower())
    return kmers

In [5]:
# Generating Dataset

def generate_dataset(df_kmers, kmer_size, max_features, split):
    # Generating kmers
    print('Generating kmers')
    kmer_dfs = []
    df_kmers['words'] = df_kmers.apply(
        lambda x: kmer_split(x['sequence'], size=kmer_size),
    axis=1)
    df_kmers = df_kmers.drop('sequence', axis=1)
    df_kmers.head()
    print('kmers generated')

    # Splitting into train/test data
    print('Splitting into train/test')
    perm = np.random.permutation(len(df_kmers))

    train_data, test_data = train_test_split(df_kmers, test_size=split)
    train_kmers = []
    for cur_kmer_list in tqdm(train_data['words'].values, desc='Loading kmers'):
        train_kmers.extend(cur_kmer_list)
    print('Split into train/test')

    # Create vectorizer
    print('Creating vectorizer')
    vectorizer = CountVectorizer(
        max_features=max_features,
    ).fit(train_kmers)
    print('Vectorizer created')

    # Ensure an even split
    print('Train:\n', train_data['label'].value_counts())
    print('Test:\n', test_data['label'].value_counts())

    # Vectorize kmers
    print('Vectorizing kmers')
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    print('Transforming train data')
    for data, label in tqdm(zip(train_data['words'], train_data['label']), total=len(train_data['label']), desc='Creating train dataset'):
        transformed = vectorizer.transform(data)
        X_train.append(transformed.toarray().sum(axis=0))
        y_train.append(label)
    print('Done transforming train data')
    print('Transforming test data')
    for data, label in tqdm(zip(test_data['words'], test_data['label']), total=len(test_data['label']), desc='Creating test dataset'):
        transformed = vectorizer.transform(data)
        X_test.append(transformed.toarray().sum(axis=0))
        y_test.append(label)
    print('Done transforming test data')

    return X_train, y_train, X_test, y_test

In [6]:
# Encoding into text

with open('accessible_small.fasta', 'r') as file:
  a = file.readlines()

a = [x.strip() for x in a]

accessible_strings = []
accessible_y_strings = []
for x in tqdm(a, desc='Loading accessible...'):
  if x[0] == '>':
    accessible_strings.append([])
    accessible_y_strings.append([1])
  else:
    accessible_strings[-1] = x

with open('notaccessible_small.fasta', 'r') as file:
  a = file.readlines()

a = [x.strip() for x in a]

notaccessible_strings = []
notaccessible_y_strings = []
for x in tqdm(a, desc='Loading not accessible...'):
  if x[0] == '>':
    notaccessible_strings.append([])
    notaccessible_y_strings.append([0])
  else:
    notaccessible_strings[-1] = x

Loading accessible...:   0%|          | 0/11925 [00:00<?, ?it/s]

Loading not accessible...:   0%|          | 0/116285 [00:00<?, ?it/s]

In [7]:
# Merge data
X_strings = np.array(accessible_strings + notaccessible_strings)
y_strings = np.array(accessible_y_strings + notaccessible_y_strings)[:,0]
X_strings.shape, y_strings.shape

((25642,), (25642,))

In [8]:
# Create a train dataframe
df_seqs = pd.DataFrame({
    'sequence': X_strings,
    'label': y_strings
})
df_seqs.head()

,sequence,label
0,TGTTAGGCAGGATCGTTTTTCCTCTGGGGCAAGATCAAAATCCAGG...,1
1,TGTGAATGGAATTCCTGACTCCATTCAGGAGTAGCTGGGGGGAGGG...,1
2,GCTCTAGGCCAATGTGAAACCCGTGGTACCGGAGCCCCAAACAAAC...,1
3,TGAATCCATTAATTAGTCCCAGGTACAGACTTGCTGCTGGACTTGA...,1
4,TCCCTAGGACAAAAAGGCAATTGTCAACTTTTAAGGAAGGAGCCGA...,1


In [9]:
df_kmers = df_seqs.copy()

In [10]:
# Values to generate dataset
kmer_size = 20
max_features = 20000
split=0.2

X_train, y_train, X_test, y_test = generate_dataset(
    df_kmers=df_kmers,
    kmer_size=kmer_size,
    max_features=max_features,
    split=split
)

Generating kmers
kmers generated
Splitting into train/test


Loading kmers:   0%|          | 0/20513 [00:00<?, ?it/s]

Split into train/test
Creating vectorizer
Vectorizer created
Train:
 0    18614
1     1899
Name: label, dtype: int64
Test:
 0    4643
1     486
Name: label, dtype: int64
Vectorizing kmers
Transforming train data


Creating train dataset:   0%|          | 0/20513 [00:00<?, ?it/s]

Done transforming train data
Transforming test data


Creating test dataset:   0%|          | 0/5129 [00:00<?, ?it/s]

Done transforming test data


### Training Model

In [38]:
# Create dataloaders
batch_size = 16

# Remeber to convert data to floats
train_data = data_utils.TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train))
train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data=data_utils.TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test))
test_loader=data_utils.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [14]:
# Features and labels
num_features=len(train_data[0][0])
num_labels=2

In [15]:
# Our dataset
train_loader.dataset

In [16]:
# Find shape of input data
train_loader.dataset[0][0].shape

torch.Size([20000])

In [39]:
# Find type of input data
train_loader.dataset[0][0].dtype

torch.float32

In [43]:
# Show that our vectorization worked fr
for i in range(10):
    print(train_loader.dataset[i][0].sum(), '-', train_loader.dataset[i][1])

AttributeError: ignored

In [48]:
# Count types of accessible to non-accessible
print(f'Train proportion of non-accessible sequences: {1-sum(y_train)/len(y_train)}')
print(f'Train proportion of accessible sequences: {sum(y_train)/len(y_train)}')
print(f'Test proportion of non-accessible sequences: {1-sum(y_test)/len(y_test)}')
print(f'Test proportion of accessible sequences: {sum(y_test)/len(y_test)}')

Train proportion of non-accessible sequences: 0.9074245600350996
Train proportion of accessible sequences: 0.09257543996490031
Test proportion of non-accessible sequences: 0.9052446870735036
Test proportion of accessible sequences: 0.09475531292649639


# Model

## Architecture

In [18]:
class CNN(nn.Module):

    # def __init__(self, max_features):
    #     super(CNN, self).__init__()
    #     self.network = nn.Sequential(
    #         nn.Conv1d(max_features, 1000, kernel_size=3, padding=1), # Output (None, 1000, 128), Param# 3200
    #         nn.ReLU(),
    #         nn.MaxPool1d(3, 2), # Output: 500
    #         nn.Conv1d(500, 500, kernel_size=3), # Output: 500
    #         nn.ReLU(),
    #         nn.MaxPool1d(3, 2), # Output: 250
    #         nn.Flatten(),
    #         nn.Linear(250, 128),
    #         nn.ReLU(),
    #         nn.Linear(128, 64),
    #         nn.ReLU(),
    #         nn.Linear(64, 6),
    #         nn.ReLU(),
    #         nn.Linear(6, 2)
    #     )

    def __init__(self, max_features, n_hidden_1, n_hidden_2):
        super(CNN, self).__init__()
        self.max_features = max_features
        self.n_hidden_1 = n_hidden_1
        self.n_hidden_2 = n_hidden_2
        self.network = nn.Sequential(
            nn.Linear(max_features, n_hidden_1),
            nn.ReLU(True),
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True),
            nn.Linear(n_hidden_2, 2),
            nn.ReLU(True)
        )

    def forward(self, x):
        """Pass input x through the network to obtain an output"""
        # Layer 1
        x = self.network(x)
        return x

    def size(self):
        """Count the number of parameters (i.e., size of weights) in the network"""
        parameters = self.parameters()
        size = 0
        for parameter in parameters:
            size += np.prod(parameter.shape)
        return size

In [20]:
n_hidden_1 = max_features//5
n_hidden_2 = max_features//10
network = CNN(max_features, n_hidden_1, n_hidden_2)
network.to(device)  # if we are using GPU, put the network weights on the GPU

CNN(
  (network): Sequential(
    (0): Linear(in_features=20000, out_features=4000, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=4000, out_features=2000, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=2000, out_features=2, bias=True)
    (5): ReLU(inplace=True)
  )
)

In [26]:
# Check network summary
summary(network, train_loader.dataset[0][0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 4000]      80,004,000
              ReLU-2                 [-1, 4000]               0
            Linear-3                 [-1, 2000]       8,002,000
              ReLU-4                 [-1, 2000]               0
            Linear-5                    [-1, 2]           4,002
              ReLU-6                    [-1, 2]               0
Total params: 88,010,002
Trainable params: 88,010,002
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.08
Forward/backward pass size (MB): 0.09
Params size (MB): 335.73
Estimated Total Size (MB): 335.90
----------------------------------------------------------------


## Tuning Model
Note: too sleepy to split into training/validation rn but fix it later

In [32]:
# Hyperparameters
learning_rate = 5e-3
epoch_max = 35
weight_decay = 1e-5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(network.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [33]:
@torch.no_grad()  # we don't want to compute gradients here!
def evaluate(loader, network, criterion):
    network.eval()  # put network into evaluation mode (mostly for normalization)
    losses = []
    accuracies = []
    for inputs, labels in loader:
        inputs = inputs.to(device)  # put inputs and labels on GPU (if it is available)
        labels = labels.to(device)
        outputs = network(inputs)  # pass inputs through network to get outputs
        loss = criterion(outputs, labels)  # evaluate outputs with criterion to get loss
        accuracy = (torch.max(outputs, dim=1)[1] == labels).to(torch.float32).mean()  # accuracy
        losses.append(loss.cpu().numpy())
        accuracies.append(accuracy.cpu().numpy())
    return np.mean(losses), np.mean(accuracies)

In [40]:
# Train model
results = {"train losses": [], "train accuracies": [], "test losses": [], "test accuracies": []}

for epoch in tqdm(range(epoch_max)):
    for inputs, labels in tqdm(train_loader, leave=False):

        # put inputs and labels on GPU (if it is available)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # put network into training mode (mostly for batch normalization)
        network.train()
        # zero-out gradients
        optimizer.zero_grad()
        # pass inputs forward through network to get outputs
        outputs = network(inputs)
        # evaluate outputs with criterion to get loss
        loss = criterion(outputs, labels)
        # backpropagate through loss to compute gradients (loss w.r.t. weights)
        loss.backward()
        # use gradients to perform optimization (e.g., NAdam)
        optimizer.step()

    # after every epoch, evaluate results on the entire training & testing set (slow)
    train_loss, train_accuracy = evaluate(train_loader, network, criterion)
    test_loss, test_accuracy = evaluate(test_loader, network, criterion)
    # store results
    results["train losses"].append(train_loss)
    results["train accuracies"].append(train_accuracy)
    results["test losses"].append(test_loss)
    results["test accuracies"].append(test_accuracy)
    # print results so we can see how the network is doing
    result = f"{('[' + str(epoch + 1) + ']'):5s}   "\
                f"Train: {str(train_accuracy * 100):.6}% accuracy ({str(train_loss):.6} loss)   "\
                f"Test: {str(test_accuracy * 100):.6}% accuracy ({str(test_loss):.6} loss)"
    tqdm.write(result)

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1283 [00:00<?, ?it/s]

[1]     Train: 90.749% (0.2843)   Test: 90.537% (0.3459)


  0%|          | 0/1283 [00:00<?, ?it/s]

[2]     Train: 90.749% (0.1589)   Test: 90.507% (0.3625)


...

[4]     Train: 90.749% (0.3376)   Test: 90.507% (0.6931)


  0%|          | 0/1283 [00:00<?, ?it/s]

[5]     Train: 90.749% (0.1570)   Test: 90.537% (0.3347)


  0%|          | 0/1283 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# etc

### Planning

In [ ]:
def kmer_split(seq, size=5):
    kmers = []
    for x in range(len(seq) - size + 1):
        kmers.append(seq[x:x+size].lower())
    return kmers

In [ ]:
# Encoding into text

with open('data/accessible.fasta', 'r') as file:
  a = file.readlines()

a = [x.strip() for x in a]

accessible_strings = []
accessible_y_strings = []
for x in tqdm(a, desc='Loading accessible...'):
  if x[0] == '>':
    accessible_strings.append([])
    accessible_y_strings.append([1])
  else:
    accessible_strings[-1] = x

with open('data/notaccessible.fasta', 'r') as file:
  a = file.readlines()

a = [x.strip() for x in a]

notaccessible_strings = []
notaccessible_y_strings = []
for x in tqdm(a, desc='Loading not accessible...'):
  if x[0] == '>':
    notaccessible_strings.append([])
    notaccessible_y_strings.append([0])
  else:
    notaccessible_strings[-1] = x

Loading not accessible...: 100%|██████████| 2392245/2392245 [00:03<00:00, 701801.90it/s] 


In [ ]:
X_strings = np.array(accessible_strings + notaccessible_strings)
y_strings = np.array(accessible_y_strings + notaccessible_y_strings)[:,0]

In [ ]:
len(accessible_strings), len(accessible_y_strings)

(47239, 47239)

In [ ]:
X_strings.shape, y_strings.shape

((525688,), (525688,))

In [ ]:
print(X_strings[0])
print(y_strings[0])

TGTTAGGCAGGATCGTTTTTCCTCTGGGGCAAGATCAAAATCCAGGTCCT
1


In [ ]:
# Split into train/test datasets
# X_train, X_test, y_train, y_test = train_test_split(X_strings, y_strings, test_size=0.2)

In [ ]:
# Create a train dataframe
df_seqs = pd.DataFrame({
    'sequence': X_strings,
    'label': y_strings
})

In [ ]:
df_seqs.head()

,sequence,label
0,TGTTAGGCAGGATCGTTTTTCCTCTGGGGCAAGATCAAAATCCAGG...,1
1,TGTGAATGGAATTCCTGACTCCATTCAGGAGTAGCTGGGGGGAGGG...,1
2,GCTCTAGGCCAATGTGAAACCCGTGGTACCGGAGCCCCAAACAAAC...,1
3,TGAATCCATTAATTAGTCCCAGGTACAGACTTGCTGCTGGACTTGA...,1
4,TCCCTAGGACAAAAAGGCAATTGTCAACTTTTAAGGAAGGAGCCGA...,1


In [ ]:
df_kmers = df_seqs.copy()

In [ ]:
# Values to generate dataset
kmer_size = 9
max_features = 1500
split=0.2

In [ ]:
# Generating kmers
kmer_dfs = []
df_kmers['words'] = df_kmers.apply(
    lambda x: kmer_split(x['sequence'], size=kmer_size),
axis=1)
df_kmers = df_kmers.drop('sequence', axis=1)
df_kmers.head()

,label,words
0,1,"[tgttaggca, gttaggcag, ttaggcagg, taggcagga, a..."
1,1,"[tgtgaatgg, gtgaatgga, tgaatggaa, gaatggaat, a..."
2,1,"[gctctaggc, ctctaggcc, tctaggcca, ctaggccaa, t..."
3,1,"[tgaatccat, gaatccatt, aatccatta, atccattaa, t..."
4,1,"[tccctagga, ccctaggac, cctaggaca, ctaggacaa, t..."


In [ ]:
# Splitting into train/test data
perm = np.random.permutation(len(df_kmers))

train_data, test_data = train_test_split(df_kmers, test_size=split)
train_kmers = []
for cur_kmer_list in tqdm(train_data['words'].values, desc='Loading kmers'):
    train_kmers.extend(cur_kmer_list)
vectorizer = CountVectorizer(
    max_features=max_features,
).fit(train_kmers)

Loading kmers: 100%|██████████| 420550/420550 [00:00<00:00, 699924.15it/s]


In [ ]:
# Ensure an even split
print(train_data['label'].value_counts())
print(test_data['label'].value_counts())

0    382764
1     37786
Name: label, dtype: int64
0    95685
1     9453
Name: label, dtype: int64


In [ ]:
# Create final train/test data
X_train = []
Y_train = []
X_test = []
Y_test = []
for data, label in tqdm(zip(train_data['words'], train_data['label']), total=len(train_data['label']), desc='Creating train dataset'):
    transformed = vectorizer.transform(data)
    X_train.append(transformed.toarray().sum(axis=0))
    Y_train.append(label)
for data, label in tqdm(zip(test_data['words'], test_data['label']), total=len(test_data['label']), desc='Creating test dataset'):
    transformed = vectorizer.transform(data)
    X_test.append(transformed.toarray().sum(axis=0))
    Y_test.append(label)

Creating test dataset: 100%|██████████| 105138/105138 [00:32<00:00, 3207.96it/s]


In [ ]:
vectorizer = CountVectorizer(
    max_features=max_features,
).fit(train_kmers)

In [ ]:
d = ['tccacatct', 'ccacatcta', 'cacatctat', 'acatctatt', 'catctatta', 'atctattac', 'tctattaca', 'ctattacaa', 'tattacaat', 'attacaatc', 'ttacaatca', 'tacaatcaa', 'acaatcaaa', 'caatcaaag', 'aatcaaaga', 'atcaaagac', 'tcaaagacg', 'caaagacgc', 'aaagacgca', 'aagacgcaa', 'agacgcaac', 'gacgcaacg', 'acgcaacgt', 'cgcaacgtt', 'gcaacgttt', 'caacgtttt', 'aacgttttc', 'acgttttct', 'cgttttctt', 'gttttcttt', 'ttttctttt', 'tttcttttt', 'ttctttttt', 'tcttttttc', 'cttttttcc', 'ttttttcct', 'tttttcctc', 'ttttcctca', 'tttcctcat', 'ttcctcatt', 'tcctcattg', 'cctcattgt']
v = vectorizer.transform(d)

In [ ]:
def generate_dataset(df, kmer_size, max_features, split):
    # Generating kmers
    kmer_dfs = []
    df_kmers['words'] = df_kmers.apply(
        lambda x: kmer_split(x['sequence'], size=kmer_size),
    axis=1)
    df_kmers = df_kmers.drop('sequence', axis=1)
    df_kmers.head()

    # Splitting into train/test data
    perm = np.random.permutation(len(df_kmers))

    train_data, test_data = train_test_split(df_kmers, test_size=split)
    train_kmers = []
    for cur_kmer_list in tqdm(train_data['words'].values, desc='Loading kmers'):
        train_kmers.extend(cur_kmer_list)
    vectorizer = CountVectorizer(
        max_features=max_features,
    ).fit(train_kmers)

    # Ensure an even split
    print('Train:\n', train_data['label'].value_counts())
    print('Test:\n', test_data['label'].value_counts())

    # Vectorize kmers
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for data, label in tqdm(zip(train_data['words'], train_data['label']), total=len(train_data['label']), desc='Creating train dataset'):
        transformed = vectorizer.transform(data)
        X_train.append(transformed.toarray().sum(axis=0))
        y_train.append(label)
    for data, label in tqdm(zip(test_data['words'], test_data['label']), total=len(test_data['label']), desc='Creating test dataset'):
        transformed = vectorizer.transform(data)
        X_test.append(transformed.toarray().sum(axis=0))
        y_test.append(label)

    return X_train, y_train, X_test, y_test

<42x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

## Encoding

In [ ]:
# Encoding into numbers
map = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

with open('data/accessible.fasta', 'r') as file:
  a = file.readlines()

a = [x.strip() for x in a]

accessible = []
accessible_y = []
for x in tqdm(a, desc='Loading accessible...'):
  if x[0] == '>':
    accessible.append([])
    accessible_y.append([1])
  else:
    for l in x:
      accessible[-1].append(map[l])

with open('data/notaccessible.fasta', 'r') as file:
  a = file.readlines()

a = [x.strip() for x in a]

notaccessible = []
notaccessible_y = []
for x in tqdm(a, desc='Loading not accessible...'):
  if x[0] == '>':
    notaccessible.append([])
    notaccessible_y.append([0])
  else:
    for l in x:
      notaccessible[-1].append(map[l])

Loading not accessible...: 100%|██████████| 2392245/2392245 [00:23<00:00, 100005.49it/s]


In [ ]:
X = np.array(accessible + notaccessible)
y = np.array(accessible_y + notaccessible_y)

In [ ]:
X.shape, y.shape

((525688, 200), (525688, 1))

In [ ]:
# Split into train/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.shape, y_train.shape

((420550, 200), (420550, 1))